Install basic dependencies for running NasBench

In [ ]:
%cd machop/chop/NASLib

In [ ]:
%pip install -e . --quiet
%pip install ConfigSpace==0.6 --quiet

In [ ]:
# For running cifar10
!source scripts/bash_scripts/download_benchmarks.sh nb201 cifar10

'/home/ansonhon/mase_project/machop/chop/NASLib'